<a href="https://colab.research.google.com/github/Mohammed-Jemal/DSC205/blob/main/DSC205_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Project Title: "Analyzing Educational Disparities in Student Performance

In [ ]:
!pip install -q streamlit
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
# import subprocess
# subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"])
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00
--2023-12-13 17:11:00--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2023.10.0/cloudflared-linux-amd64 [following]
--2023-12-13 17:11:00--  https://github.com/cloudflare/cloudflared/releases/download/2023.10.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting res

In [ ]:
!grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1 | xargs -I {} echo "Your tunnel url {}"

Your tunnel url https://denmark-reasonable-enb-usc.trycloudflare.com


In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv('/content/drive/MyDrive/DS101/StudentsPerformance.csv')

# Title of the app
st.title('Student Performance Analysis')

st.set_option('deprecation.showPyplotGlobalUse', False)
st.subheader('Score Distribution by Lunch Type: A Boxplot Analysis')
def plote(df: pd.DataFrame, subject: str):
    plt.figure(figsize=(12, 8))

    if subject == 'math':
        y_col = 'math score'
        title = 'Math'
    elif subject == 'writing':
        y_col = 'writing score'
        title = 'Writing'
    elif subject == 'reading':
        y_col = 'reading score'
        title = 'Reading'
    else:
        raise ValueError("Invalid subject. Use 'math', 'writing', or 'reading'.")

    ax = sns.boxplot(x='lunch', y=y_col, data=df)
    plt.axhline(y=df[y_col].median(), color='r', linestyle='--', label=f'Median {title} Score: {df[y_col].median():.2f}')
    plt.legend()
    plt.title(f'How do lunch types affect {title} scores?', wrap=True)
    st.pyplot()

# Sidebar with radio buttons
selected_subject = st.sidebar.radio("Select the subject to explore effect Lunch", ['math', 'writing', 'reading'])

# Plot based on the selected subject
plote(df, selected_subject)



def plote(df:pd.DataFrame):
  st.subheader('Correlation Between the scores')
  Correlation = df[['math score','reading score','writing score']].corr()
  fig_heatmap=plt.figure(figsize=(8,6))
  sns.heatmap(Correlation,annot=True,cmap='coolwarm',fmt='.2f')
  st.pyplot(fig_heatmap)

plote(df)
st.subheader('Scores Across Parental Education Levels: Barchart Analysis')
def plote(df: pd.DataFrame, score_type: str):
    sns.set(style="whitegrid")
    plt.figure(figsize=(12, 8))

    if score_type == 'math':
        y_col = 'math score'
        title = 'Math'
    elif score_type == 'writing':
        y_col = 'writing score'
        title = 'Writing'
    elif score_type == 'reading':
        y_col = 'reading score'
        title = 'Reading'
    else:
        raise ValueError("Invalid score_type. Use 'math', 'writing', or 'reading'.")

    ax = sns.barplot(x="parental level of education", y=y_col, data=df, palette="Set3", ci=None)
    #ax.axhline(y=df[y_col].median(), color='r', linestyle='--', label=f'Median {title} Score: {:.2f}'.format(df[y_col].median()))
    ax.axhline(y=df[y_col].median(), color='r', linestyle='--', label='Median {} Score: {:.2f}'.format(title, df[y_col].median()))

    ax.legend(loc='best', fontsize=12)
    plt.title(f'Distribution of {title} scores among students with different levels of parental education', wrap=True, fontsize=16)
    st.pyplot()

# Sidebar with radio buttons
selected_subject = st.sidebar.radio("Select the subject to explore impact of parental eduacation", ['math', 'writing', 'reading'])

plote(df, selected_subject)

st.subheader('Distribution of Math score based on Test preparation')
def plote(data: pd.DataFrame):
    data_prep = data.copy()
    sns.set(style="ticks")
    data_prep = data_prep.dropna(subset=['math score', 'test preparation course', 'gender'])
    data_prep['test preparation course'] = data_prep['test preparation course'].map({'none': 'Not Completed', 'completed': 'Completed'})

    # Create a Streamlit figure
    fig, ax = plt.subplots(figsize=(10, 6))

    # Use sns plotting functions with ax
    sns.boxplot(x='test preparation course', y='math score', hue='gender', data=data_prep, palette='Set2', ax=ax)
    ax.axhline(y=data_prep['math score'].median(), color='r', linestyle='--', label='Median Math Score')
    ax.legend(loc='upper left')
    ax.set_title('What is the distribution of math scores among students who completed the test preparation course and those who did not, grouped by gender?')

    # Display the plot using st.pyplot()
    st.pyplot(fig)
plote(df)

Overwriting app.py
